In [ ]:
from attr.validators import instance_of
#创建集合单元
from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient

client = MilvusClient(uri="http://192.168.200.130:19530")

def create_collection():
    # 定义字段 schemas
    id_field = FieldSchema(name="id", dtype=DataType.INT64, auto_id=True, is_primary=True)
    url_field = FieldSchema(name="url", dtype=DataType.VARCHAR, max_length=65535)
    content_field = FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=65535)
    content_vector_field = FieldSchema(name="content_vector", dtype=DataType.FLOAT_VECTOR, dim=1024)
    title_field = FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=65535)
    title_vector_field = FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=1024)
    publish_time_field = FieldSchema(name="publish_time", dtype=DataType.INT64)

    # 创建 Collection schema
    schema = CollectionSchema(
        fields=[id_field, url_field, content_field, content_vector_field, title_field, title_vector_field, publish_time_field],
        description="CUMT_GPT的Chuxin数据集"
    )

    # 创建 Collection
    collection_name = "cumt_gpt_chuxin"
    if client.has_collection(collection_name):
        client.drop_collection(collection_name)

    client.create_collection(collection_name=collection_name, schema=schema)

    # 为 content_vector 创建索引
    # index_params = client.prepare_index_params()
    # index_params.add_index(
    #     field_name="content_vector",
    #     index_type="IVF_FLAT",
    #     metric_type="COSINE",
    #     params={"nlist": 128}
    # )
    
    # HNSW
    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name="content_vector",
        index_type="HNSW",
        metric_type="COSINE",
        params={"M": 32, "efConstruction": 250}
    )
    index_params.add_index(
        field_name="title_vector",
        index_type="HNSW",
        metric_type="COSINE",
        params={"M": 32, "efConstruction": 250}
    )

    client.create_index(
        collection_name=collection_name,
        index_params=index_params,
        sync=False)


create_collection()
client.list_indexes(collection_name="cumt_gpt_chuxin")

In [ ]:
# 插入数据单元
import json
import time
from datetime import datetime
from pymilvus import MilvusClient
from langchain_community.embeddings import OllamaEmbeddings
from FlagEmbedding import FlagModel
from sklearn.preprocessing import normalize
# from sonyflake import SonyFlake

client = MilvusClient(uri="http://192.168.200.130:19530")

def emb_text(texts):
    # 使用 bert-large-chinese 模型
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    
    embedding_model = FlagModel('chuxin-llm/Chuxin-Embedding', use_fp16=True)
    embedding_vectors = embedding_model.encode_queries(texts, batch_size=256)
    # embedding_vectors = normalize(embedding_vectors, norm='l2')
    
    return embedding_vectors

# 文档列表
# docs = [
#     "Artificial intelligence was founded as an academic discipline in 1956.",
#     "Alan Turing was the first person to conduct substantial research in AI.",
#     "Born in Maida Vale, London, Turing was raised in southern England.",
# ]
# doc = "矿小助是一款由FlyingStudio（中国矿业大学翔工作室）开发维护的校园软件。提供课表、考试、成绩、校车校历、图书馆藏、校卡余额、宿舍电量等查询功能；同时具有课表导入日历、加权成绩计算、校园网自动登录、个性化主题背景等实用功能。"

contents = []
titles = []

# 打开 JSON 文件
# filename = '原始数据/articles.json'
# filename = ('原始数据/kxz.json')
# filename = '原始数据/news.json'
filename = '原始数据/矿大新闻网.json'
with open(filename, 'r', encoding='utf-8') as f:
    python_data = json.load(f)
    for data in python_data['data']:
        contents.append(data.get("content"))
        titles.append(data.get("title"))

# 设置每批次处理的数据量
batch_size = 1000  # 每批次插入1000条数据

total_length = len(contents)
for start in range(0, total_length, batch_size):
    end = min(start + batch_size, total_length)
    
    # 分批生成内容向量和标题向量
    content_batch = contents[start:end]
    title_batch = titles[start:end]
    
    content_vectors = emb_text(content_batch)
    title_vectors = emb_text(title_batch)
    
    # 要插入的数据
    datas = []
    for i, data in enumerate(python_data['data'][start:end]):
        # 使用 strptime 将字符串解析为日期对象
        date_obj = datetime.strptime(data.get("date"), "%Y-%m-%d")
        
        # 使用 strftime 格式化为所需的格式，并将其转换为整数
        date_int = int(date_obj.strftime("%Y%m%d"))

        datas.append({
                # "id": sonyflake_id,
                "url": data.get("url"),
                "content": data.get("content"),
                "content_vector": content_vectors[i],
                "title": data.get("title"),
                "title_vector": title_vectors[i],
                "publish_time": date_int
            })

    # 插入当前批次的数据
    res = client.insert(collection_name="cumt_gpt_chuxin", data=datas)
    print(f"已插入第 {start // batch_size + 1} 批数据，共插入 {len(datas)} 条记录")
    time.sleep(5)  # 等待5秒


# print(res)


In [7]:
from pymilvus import AnnSearchRequest, WeightedRanker, RRFRanker
# 测试向量搜索单元
# from langchain_community.embeddings import OllamaEmbeddings
# from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient
from FlagEmbedding import FlagModel
from langchain_community.document_transformers import LongContextReorder
import numpy as np

client = MilvusClient(uri="http://192.168.200.130:19530")


def emb_text(texts):
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    # embedding_vectors = embedding_model.encode(text, batch_size=1024)
    embedding_model = FlagModel('chuxin-llm/Chuxin-Embedding', use_fp16=True)
    embedding_vectors = embedding_model.encode_queries(texts, batch_size=256)

    # embedding_vectors = normalize(embedding_vectors, norm='l2')

    return embedding_vectors


doc = ["什么时候举行庆祝矿大115周年篮球友谊赛"]

doc_vector = emb_text(doc)

search_params = {
    "metric_type": "COSINE",
    "params": {}
}

# IVF_FLAT
# res = client.search(
#     collection_name="cumt_gpt_qwen",
#     data=doc_vector,
#     limit=10,
#     output_fields=["id", "url", "content", "publish_time"],
#     search_params=search_params
# )

# HNSW
# res = client.search(
#   collection_name="cumt_gpt_chuxin", # Collection name
#   data=doc_vector, # Replace with your query vector
#   search_params={
#     "metric_type": "COSINE",
#     "params": {"ef":150, "radius":0.36}, # Search parameters
#   }, # Search parameters
#   limit=10, # Max. number of search results to return
#   output_fields=["id","url","content","publish_time"], # Fields to return in the search results
#   consistency_level="Bounded"
# )

res = client.hybrid_search(
    collection_name="cumt_gpt_chuxin",
    reqs=[
        AnnSearchRequest(
            data=doc_vector,  # 查询向量
            anns_field="content_vector",  # 搜索字段
            param={"metric_type": "COSINE", "params": {"ef": 150, "radius": 0.36}},  # 搜索参数
            limit=5,  # 返回前10个结果
            # expr="publish_time > 1627776000"  # 过滤条件
        ),
        AnnSearchRequest(
            data=doc_vector,  # 查询向量
            anns_field="title_vector",  # 搜索字段
            param={"metric_type": "COSINE", "params": {"ef": 150, "radius": 0.36}},  # 搜索参数
            limit=5,  # 返回前10个结果
            # expr="publish_time > 1627776000"  # 过滤条件
        )
    ],
    # ranker=WeightedRanker(float(0.7), float(0.3)),
    ranker=RRFRanker(),
    limit=5,
    output_fields=["id", "url", "content", "title", "publish_time"],
    timeout=60,
    consistency_level="Bounded"
    # partition_names: Optional[List[str]] = None
)

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(res[0])

print(res)
print("----------------")
print(reordered_docs)

data: ["[{'id': 453316957169527507, 'distance': 0.032522473484277725, 'entity': {'title': '“矿大杯”2010年海峡两岸篮球邀请赛开战（10.14）', 'publish_time': 20101014, 'id': 453316957169527507, 'url': 'https://news.cumt.edu.cn/../info/1002/47268.htm', 'content': '“矿大杯”2010年海峡两岸篮球邀请赛开战（10.14）作者：系统管理员发布时间：2010-10-14\\n新闻来源：体育学院摄影：责任编辑：审核:'}}, {'id': 453316957169525649, 'distance': 0.03100961446762085, 'entity': {'title': '第十六届CUBA中国大学生篮球联赛（东南赛区）在中国矿大开幕（4.22）', 'publish_time': 20140422, 'id': 453316957169525649, 'url': 'https://news.cumt.edu.cn/../info/1002/44664.htm', 'content': '第十六届CUBA中国大学生篮球联赛（东南赛区）在中国矿大开幕（4.22）作者：系统管理员发布时间：2014-04-22\\n\\n4月21日13：00，第十六届CUBA中国大学生篮球联赛（东南赛区）在中国矿业大学文昌校区体育馆开幕。\\n中国大学生体育协会专职副主席张燕军、王晓毅，江苏省教育厅副巡视员歩锦昆，江苏省教育厅体卫艺处处长杜伟，中国矿业大学校党委书记邹放鸣、副校长宋学锋，中国大学生体育协会篮球分会秘书长刘庆广，江苏徐州医药高等职业学校副校长袁龙，徐州市体育局局长、党委书记李跃华，李宁集团公司代表焦阳，杭州顶益食品有限公司华东区总经理孙百弘，本次比赛裁判长、国际级裁判杨茂功等出席开幕式。5000余名CUBA球迷参加了开幕式。开幕式由中国矿业大学副校长宋学锋主持。\\n中国矿业大学校党委书记邹放鸣致欢迎辞。他代表学校向各兄弟院校的领队、教练、运动员和各位嘉宾致以热烈的欢迎，并向关心和支持此次大赛的各位领导、有关部门及社会各界表示衷心的感谢。他表示，加强